<a href="https://colab.research.google.com/github/Philxen/Spoon-Knife-trucho/blob/main/modelSelection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import des bibliothèques nécessaires

!pip install imblearn

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import pickle
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import numpy as np
import pandas as pd
from sklearn.exceptions import DataConversionWarning
import warnings
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

In [3]:
with open('/content/sample_data/df_NC_tfidf_processed.txt','rb') as fich:
    mon_depickler = pickle.Unpickler(fich)
    df_transformed = mon_depickler.load()
fich.closed

True

In [5]:
df_transformed[0].head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.108222,0.132043,-0.176247,-0.063984,0.048009,0.046303,-0.140499,-0.022533,0.014784,-0.032771,...,-0.050079,-0.002366,0.013881,0.044899,-0.013551,0.007734,0.019178,-0.032607,-0.024614,0.004113
1,-0.057872,-0.020792,-0.140899,-0.045695,0.136261,-0.009008,0.052231,0.127060,0.003511,0.075291,...,0.027502,-0.018362,0.034206,0.002145,0.076570,-0.067452,-0.063715,-0.001893,-0.000267,-0.001756
2,-0.067734,0.039166,-0.025912,-0.006927,-0.022884,0.077834,-0.020051,0.041596,0.045967,-0.007923,...,0.035392,0.034142,0.010414,0.001385,0.003735,-0.052750,-0.031240,0.075648,-0.026184,-0.014616
3,-0.036329,-0.075801,0.023502,-0.062447,-0.027868,0.128359,-0.007818,-0.044972,-0.109764,0.083500,...,-0.015326,-0.085157,-0.019224,-0.039886,0.008692,0.058055,0.023224,-0.002058,-0.020334,-0.031797
4,-0.032379,-0.027507,0.000548,0.004713,-0.044699,0.080827,-0.004600,-0.020317,-0.056274,-0.013775,...,0.009578,-0.018172,-0.007938,-0.015172,-0.004005,0.042362,0.008790,0.018272,0.008284,-0.028766


In [6]:
# Diviser le dataset en features (X) et target (y)
X = df_transformed[0]
y = df_transformed[1]

# Diviser les données en ensemble d'entraînement et ensemble de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# équilibrage des données de test
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [7]:
# Définition des types de classifieurs à comparer

clf_lr = LogisticRegression(random_state=22, max_iter=2000)
clf_rf = RandomForestClassifier(random_state=22)
clf_svc = SVC(random_state=22)


param_grid_lr = {'solver': ['liblinear', 'lbfgs'], 'C': np.logspace(-4, 2, 9)}

param_grid_rf = [{'n_estimators': [10, 50, 100, 250, 500, 1000],
                  'min_samples_leaf': [1, 3, 5],
                  'max_features': ['sqrt', 'log2']}]

param_grid_svc = [{'kernel': ['rbf'], 'C': np.logspace(-4, 4, 9), 'gamma': np.logspace(-4, 0, 4)},
                  {'kernel': ['linear'], 'C': np.logspace(-4, 4, 9)}]

In [8]:
# Pour chaque type de classifieur , instanciation d'un comparateurs/sélectionneurs

gridcvs = {}

for pgrid, clf, name in zip((param_grid_rf, param_grid_svc, param_grid_lr),
                            (clf_rf, clf_svc, clf_lr),
                            ('RF', 'SVM', 'LogisticRegression')):
    gcv = GridSearchCV(clf, pgrid, cv=3, refit=True)
    gridcvs[name] = gcv

In [9]:
# Sélection du type de classifier ayant la meilleure moyenne (et moindre écart type)

outer_cv = StratifiedKFold(n_splits=3, shuffle=True)
outer_scores = {}

for name, gs in gridcvs.items():
    nested_score = cross_val_score(gs, X_train_resampled, y_train_resampled, cv=outer_cv)
    outer_scores[name] = nested_score
    print(f'{name}: outer accuracy {100*nested_score.mean():.2f} +/- {100*nested_score.std():.2f}')

KeyboardInterrupt: 

In [10]:
!nvidia-smi

Tue Mar 26 09:12:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              24W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Tirer pleinement parti de votre abonnement à Colab



## GPU plus rapides

<p>Les utilisateurs ayant souscrit l'un des forfaits payants de Colab ont accès à des GPU premium. Vous pouvez mettre à niveau les paramètres GPU de votre notebook via <code>Environnement d'exécution &gt; Modifier le type d'exécution</code> dans le menu pour activer l'accélérateur Premium. En fonction des disponibilités, sélectionner un GPU premium peut vous donner accès à un GPU Nvidia V100 ou A100.</p>
<p>La version sans frais de Colab permet d'accéder aux GPU T4 de Nvidia, selon les restrictions de quota et la disponibilité.</p>

Vous pouvez voir le GPU qui vous est attribué à tout moment en exécutant la cellule suivante. Si le résultat d'exécution de la cellule de code ci-dessous est "Not connected to a GPU" &#40;"Non connecté à un GPU"&#41;, vous pouvez modifier l'environnement d'exécution en sélectionnant <code>Environnement d'exécution &gt; Modifier le type d'exécution</code> dans le menu pour activer un accélérateur GPU. Ensuite, réexécutez la cellule de code.

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Pour utiliser un GPU avec votre notebook, sélectionnez <code>Environnement d'exécution &gt; Modifier le type d'exécution</code>, puis dans le menu déroulant "Accélérateur matériel", sélectionnez l'option "GPU".

## Plus de mémoire

Les utilisateurs ayant souscrit l'un des forfaits payants de Colab ont accès aux VM à haute capacité de mémoire dès qu'elles sont disponibles.
Vous pouvez surveiller la quantité de mémoire à votre disposition à tout moment en exécutant la cellule de code suivante. Si le résultat d'exécution de la cellule de code ci-dessous est "Not using a high-RAM runtime" &#40;"Un environnement d'exécution à mémoire RAM élevée n'est pas utilisé"&#41;, vous pouvez activer un environnement d'exécution à mémoire RAM élevée en sélectionnant <code>Environnement d'exécution &gt; Modifier le type d'exécution</code> dans le menu. Ensuite, sélectionnez "Mémoire RAM élevée" dans le menu déroulant "Configuration de l'environnement d'exécution", puis réexécutez la cellule de code.

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

## Environnements d'exécution plus durables

Tous les environnements d'exécution Colab sont réinitialisés après un certain temps &#40;plus rapidement s'il ne s'agit pas d'un environnement d'exécution de code&#41;. Les utilisateurs de Colab Pro et Colab Pro+ ont accès à des environnements d'exécution plus durables que les utilisateurs de la version sans frais de Colab.

## Exécution en arrière-plan

Les utilisateurs de Colab Pro+ ont accès à l'exécution en arrière-plan, qui permet aux notebooks de poursuivre leur exécution même après la fermeture d'un onglet de navigateur. Cette option est toujours activée dans les environnements d'exécution Pro+ tant que des unités de calcul sont disponibles.


## Assouplir les limites de ressources dans Colab Pro

Vos ressources ne sont pas illimitées dans Colab. Pour profiter pleinement de Colab, évitez d'utiliser des ressources lorsque vous n'en avez pas besoin. Par exemple, n'utilisez un GPU que si nécessaire et fermez les onglets Colab lorsque vous avez terminé.

Si vous êtes confronté à des limites, vous pouvez les assouplir en achetant davantage d'unités de calcul via le <a href="https://colab.research.google.com/signup">paiement à l'usage</a>. Tout le monde peut le faire, aucun abonnement n'est requis.

## Envoyer vos commentaires

<p>Si vous avez des commentaires, n'hésitez pas à nous les faire parvenir. Pour cela, il vous suffit de sélectionner Aide &gt; Envoyer des commentaires. Si vous faites l'objet de limites d'utilisation dans Colab Pro, envisagez de vous abonner à Colab Pro+.</p>
<p>Si vous rencontrez des erreurs ou d'autres problèmes de facturation &#40;paiements&#41; avec Colab Pro, Colab Pro+ ou le paiement à l'usage, veuillez contacter <a href="mailto:colab-billing@google.com">colab-billing@google.com</a>.</p>

## Autres ressources

### Utiliser les notebooks dans Colab
- [Présentation de Colaboratory](/notebooks/basic_features_overview.ipynb)
- [Guide de Markdown](/notebooks/markdown_guide.ipynb)
- [Importer des bibliothèques et installer des dépendances](/notebooks/snippets/importing_libraries.ipynb)
- [Enregistrer et charger des notebooks dans GitHub](https://colab.research.google.com/github/googlecolab/colabtools/blob/main/notebooks/colab-github-demo.ipynb)
- [Formulaires interactifs](/notebooks/forms.ipynb)
- [Widgets interactifs](/notebooks/widgets.ipynb)

<a name="working-with-data"></a>
### Utiliser les données
- [Chargement de données : Drive, Sheets et Google Cloud Storage](/notebooks/io.ipynb)
- [Graphiques : visualiser les données](/notebooks/charts.ipynb)
- [Premiers pas avec BigQuery](/notebooks/bigquery.ipynb)

### Cours d'initiation au Machine Learning
Vous trouverez ci-dessous quelques-uns des notebooks de la formation Google en ligne sur le machine learning. Consultez la <a href="https://developers.google.com/machine-learning/crash-course/">formation complète en ligne</a> pour en savoir plus.
- [Présentation du DataFrame pandas](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/pandas_dataframe_ultraquick_tutorial.ipynb)
- [Régression linéaire avec tf.keras et des données synthétiques](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/linear_regression_with_synthetic_data.ipynb)


<a name="using-accelerated-hardware"></a>
### Utiliser le matériel accéléré
- [TensorFlow avec des GPU](/notebooks/gpu.ipynb)
- [TensorFlow avec des TPU](/notebooks/tpu.ipynb)

<a name="machine-learning-examples"></a>

## Exemples de machine learning

Pour voir des exemples complets d'analyses interactives de machine learning rendues possibles par Colaboratory, consultez ces tutoriels utilisant des modèles issus de <a href="https://tfhub.dev">TensorFlow Hub</a>.

Voici quelques exemples :

- <a href="https://tensorflow.org/hub/tutorials/tf2_image_retraining">Recyclage d'un classificateur d'images</a> : construisez un modèle Keras sur un classificateur d'images pré-entraîné pour faire la distinction entre différentes fleurs.
- <a href="https://tensorflow.org/hub/tutorials/tf2_text_classification">Classification de texte</a> : classez des avis sur des films provenant de la base de données IMDB comme <em>positifs</em> ou <em>négatifs</em>.
- <a href="https://tensorflow.org/hub/tutorials/tf2_arbitrary_image_stylization">Transfert de style</a> : utilisez le deep learning pour transférer un style d'une image à une autre.
- <a href="https://tensorflow.org/hub/tutorials/retrieval_with_tf_hub_universal_encoder_qa">Questions/Réponses sur l'encodeur de phrases universel multilingue</a> : utilisez un modèle de machine learning afin de répondre aux questions issues de l'ensemble de données SQuAD.
- <a href="https://tensorflow.org/hub/tutorials/tweening_conv3d">Interpolation vidéo</a> : prédisez ce qui s'est passé entre la première et la dernière image d'une vidéo.
